In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats as st
from scipy.stats import norm
from scipy.stats import t
import math as mth
import plotly.express as px
import seaborn as sns
import psycopg2
from psycopg2 import extras
import numpy as np
import itertools
from itertools import combinations
from collections import Counter

### Подготовим файл с данными по продажам курсов в разрезе пользователей (купивших более одного курса)
SQL-запрос:  

WITH more_than_one  AS  
(  
SELECT   
    carts.user_id,  
    count (items.resource_id)  
FROM final.cart_items  AS  items  
    JOIN final.carts ON items.cart_id = carts.id  
WHERE resource_type = 'Course' AND state = 'successful'  
GROUP BY 1  
HAVING count (distinct items.resource_id) > 1  
)  
SELECT   
   
    distinct more_than_one.user_id,  
    items.resource_id  
FROM more_than_one  
    JOIN final.carts ON more_than_one.user_id = carts.user_id  
    JOIN final.cart_items  AS items ON items.cart_id = carts.id  
    WHERE resource_type = 'Course' AND state = 'successful'  
ORDER BY 1,2  

Чтобы в дальнейшем работать с данными и провести рекомендации клиентам 


In [2]:
#Подготовили файл с данными по продажам курсов в разрезе пользователей (купивших более одного курса)
def getmore_than_one():
    query = '''WITH more_than_one  AS
(
SELECT 
    carts.user_id,
    count (items.resource_id)
FROM final.cart_items  AS  items
    JOIN final.carts ON items.cart_id = carts.id
WHERE resource_type = 'Course' AND state = 'successful' 
GROUP BY 1
HAVING count (distinct items.resource_id) > 1
)
SELECT 
   
    distinct more_than_one.user_id,
    items.resource_id
FROM more_than_one
    JOIN final.carts ON more_than_one.user_id = carts.user_id
    JOIN final.cart_items  AS items ON items.cart_id = carts.id
    WHERE resource_type = 'Course' AND state = 'successful'
ORDER BY 1,2
 '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
d = pd.DataFrame(getmore_than_one())

### Далее выявляем сколько пар различных курсов встречаются в покупках клиентов и выявить самую популярную пару курсов

In [3]:
pair = d.groupby('user_id')['resource_id'].apply(lambda x: list(np.unique(x))).reset_index()
pairs = pd.DataFrame(pair['resource_id'].apply(lambda x: list(combinations(x, 2))))
pairs_list = pairs['resource_id'].to_list()
list_pair =[]
for item in pairs_list:
    for i in item:
        list_pair.append(i)
            
set_pair = set(list_pair)
len(set_pair) # Сколько различных пар курсов встречаются вместе в покупках клиентов: 3989
count_pair = Counter(list_pair)
count_pair.most_common(1) #самая популярная пара курсов.
# в итоге мы узнали, что самая популярная пара курсов - ID 551 и ID 566, встречаются 797 раз в покупках клиентов

       

[((551, 566), 797)]

In [4]:
# Сформируем таблицу, для более удобного просмотра
df_pair_cnt = pd.DataFrame.from_dict(count_pair, orient='Index').reset_index()
df_pair_cnt = df_pair_cnt.rename(columns={'index':'pair',0:'count'})
df_pair_cnt = df_pair_cnt.sort_values(by='count',ascending=False).reset_index(drop=True)
df_pair_cnt.describe().round()
# из этой таблицы можно увидеть, что максимальное количество купленных курсов составляет 797, количество различных курсов составляет 3989, 
# самая популярная пара курсов ID 551 и 566

,count
count,3989.0
mean,10.0
std,26.0
min,1.0
25%,1.0
50%,3.0
75%,9.0
max,797.0


In [5]:
#Минимальную границу для рекомендации курсов берем выше третьего квартиля, то есть больше 9, так как выборка остается достаточно большой,
#но и исключим при этом редко встречающиеся пары курсов
#Самые популярные курсы
df_pair_pop = df_pair_cnt[df_pair_cnt['count']> 9]


In [6]:
#Уникальные курсы в корзине
course_unique = d['resource_id'].sort_values().unique().tolist()
len(course_unique) # 126

126

### Запрос SQL - 127 уникальных курсов, что отличается от уникальных курсов в корзине -126, 
это связано с тем, что по ТЗ для рекомендаций нужно использовать только те курсы, которые приобретались клиентами.
На этапе сбора данных учитывались все курсы, в том числе и не приобретенные, но которые находились в "корзине"  
--select  
--count(DISTINCT resource_id)  
--from final.cart_items  
--where resource_type = 'Course'   

In [7]:

df_courses = pd.read_csv('./unique_course.csv')
courses_list = df_courses['resource_id'].sort_values().to_list()
len(courses_list)


127

In [8]:
#Опеределяем ID курса, который не приобретался
not_pair_course = list(set(courses_list) - set(course_unique))
not_pair_course

[772]

In [9]:
#Чтобы написать рекомендации, нам нужно определить, какие курсы самые популярные

pop_courses = pd.DataFrame(d.groupby('resource_id')['user_id'].count()).reset_index()
pop_courses = pop_courses.sort_values(by='user_id', ascending=False).reset_index()
pop_courses_list = pop_courses['resource_id'][:2].to_list()
pop_courses_list

[551, 566]

In [15]:
#Создадим таблицу с рекомендациями, включая тот курс, который не приобретался
data= {'course':[not_pair_course[0]], 'recom_1':[pop_courses_list[0]], 'recom_2': [pop_courses_list[1]]}
df_recome = pd.DataFrame(data)
df_recome

,course,recom_1,recom_2
0,772,551,566


In [11]:
#Функция рекомендаций для курса
def recom(course):
    rec_list = []
    for i in count_pair.keys():
        if i[0] == course:
            rec_list.append((i, count_pair[i]))  
        elif i[1] == course:
            rec_list.append((i, count_pair[i])) 
    rec_list = sorted(rec_list, key = lambda x:x[1],reverse= True )  
    return rec_list[:2]  
recom(490)
        

[((490, 566), 253), ((490, 551), 247)]

In [12]:
#Установим лимит, так как данные берем третьего квартиля(75%) - 9.
limits = np.percentile(df_pair_cnt['count'],75)
limits

9.0

In [28]:
df_table = pd.DataFrame()
for i in course_unique:
    course = i
    if recom(i)[0][1] > limits:
        if recom(i)[0][0][0] == course:
            recom_1 = recom(i)[0][0][1]
        else:
            recom_1 = recom(i)[0][0][0]
    elif recom(i)[1][1] > limits:
        if recom(i)[1][0][0] == course:
            recom_2 = recom(i)[1][0][1]
        else:
            recom_2 = recom(i)[1][0][0]
    else:
        recom_1 = pop_courses_list[0]
        recom_2 = pop_courses_list[0]
    df_recom = pd.DataFrame({'course':[course], 'recom_1':[recom_1], 'recom_2':[recom_2]})
    df_table = pd.concat([df_table, df_recom], ignore_index=True) 
df_table    

,course,recom_1,recom_2
0,356,571,566
1,357,571,566
2,358,570,566
3,359,570,566
4,360,745,566
...,...,...,...
121,1188,1141,551
122,1198,551,551
123,1199,551,551
124,1200,551,551


Итоговая рекомендательная таблица

## Вывод
Так как пара курсов ID 551 и ID 566 являются самыми популярными в продажах, по сравнению с остальными, то можно их рекомендовать для клиентов к приобретению? но чаще во второй рекомендации.Выборка делалась на основании третьего квартиля.
Скорее всего, курс с ID 792 стоит исключить совсем из списка предлагаемых курсов, либо предложить его в качестве дополнительного курса к приобретенному, чтобы узнать, как "доработать" курс, чтобы он стал более популярным.
Рекомендации выстроены из "корзин" покупателей, "отложенных2 покупок в корзине, может из "просмотренных" вебинаров.
